In [14]:
import torch
from exp.exp_informer import Exp_Informer
from data.data_loader import Dataset_Custom
from torch.utils.data import DataLoader
import pandas as pd
import os
from utils.tools import dotdict

torch.autograd.set_detect_anomaly(True)

In [15]:
args = dotdict()
args.root_path = '../data/'
args.data_path = 'ml_features_subset.csv'

df = pd.read_csv(os.path.join(args.root_path, args.data_path))

In [16]:
import numpy as np
df = df.drop(columns= 'date')
print("Scaled data stats (mean):", df.mean())
print("Scaled data stats (std):", df.std())

Scaled data stats (mean): hdd                    7.832671e+00
cdd                    1.630863e+00
temperature            1.209819e+01
wind_speed             1.220129e+01
load_capacity_ratio    8.347124e+01
load_error             2.813177e+01
price_error           -5.377306e-02
n_spikes_30            1.248093e+00
n_spikes_45            7.618975e-01
n_spikes_60            5.236425e-01
past_da_load_error     1.353241e+06
past_da_price_error    2.772481e+04
dtype: float64
Scaled data stats (std): hdd                    7.672729e+00
cdd                    3.064689e+00
temperature            9.685664e+00
wind_speed             6.974602e+00
load_capacity_ratio    1.336311e+04
load_error             2.358071e+02
price_error            3.399242e+01
n_spikes_30            2.088551e+00
n_spikes_45            1.595518e+00
n_spikes_60            1.313896e+00
past_da_load_error     1.906557e+06
past_da_price_error    2.218108e+05
dtype: float64


In [17]:
date = df['date']
df.drop(columns= 'date', inplace= True)
df = (df - df.min()) / (df.max() - df.min() + 1e-8)
df.insert(0, 'date', value= date)
df.set_index('date', inplace= True)
df.dropna(inplace= True)
df.to_csv('../data/ml_normalized.csv')

KeyError: 'date'

In [ ]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'custom' # data
args.root_path = '../data/' # root path of data file
args.data_path = 'ml_features_subset.csv' # data file
args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'price_error' # target feature in S or MS task
args.freq = 'h' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.seq_len = 96 # input sequence length of Informer encoder
args.label_len = 48 # start token length of Informer decoder
args.pred_len = 24 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 12 # encoder input size
args.dec_in = 12 # decoder input size
args.c_out = 12 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 256 # dimension of model
args.n_heads = 4 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 1024 # dimension of fcn in model
args.dropout = 0.2 # dropout
args.attn = 'full' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = False # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.freq = 'h'

args.batch_size = 16 
args.learning_rate = 0.00001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'

In [ ]:
Exp = Exp_Informer

In [ ]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]


args.detail_freq = args.freq
args.freq = args.freq[-1:]

In [ ]:
for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

    # set experiments
    exp = Exp(args)
    
    # train
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)
    
    # test
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)

    torch.cuda.empty_cache()

Use CPU
>>>>>>>start training : informer_custom_ftM_sl96_ll48_pl24_dm256_nh4_el2_dl1_df1024_atfull_fc5_ebtimeF_dtFalse_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 18237
val 2601
test 5221


RuntimeError: Function 'MseLossBackward0' returned nan values in its 0th output.